<a href="https://colab.research.google.com/github/danielmachlab/imdb_review_classifier/blob/development/imdb_review_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB Review Classifier

### Dev Steps
1. Import Dataset from keras/tf
2. View Sentences
3. Translate encoded reviews 
4. Create Model
5. Train Model
6. Test Model
7. Tweak Hyper Parameters


In [26]:
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb

TensorFlow is already loaded. Please restart the runtime to change versions.


### View Data

In [0]:
num_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

In [28]:
word_index = imdb.get_word_index()
# shift all numbers by three to make room for placeholders
word_index = {k: (v+3) for k, v in word_index.items()} 

# add placeholders to the word_index dictionary
word_index.update({"<PAD>": 0, "<START>": 1, "<UNK>": 2, "<UNUSED>": 3})
list(word_index.items())[-10:]

[('hardboiled', 52008),
 ("voorhees'", 88586),
 ('35mm', 16818),
 ("'l'", 88587),
 ('paget', 18512),
 ('expands', 20600),
 ('<PAD>', 0),
 ('<START>', 1),
 ('<UNK>', 2),
 ('<UNUSED>', 3)]

In [29]:
# view the first review
print('sentence', x_train[0])
print('label', y_train[1])

sentence [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
label 0


### Translate Encoded Reviews

In [0]:
flipped_word_index = dict([ (v,k) for (k, v) in word_index.items() ])

In [0]:
def translate_review(review):
  return " ".join( [flipped_word_index.get(word, '?') for word in review] )

In [32]:
translate_review(x_train[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [0]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train, value=word_index["<PAD>"], padding="post", maxlen=250)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, value=word_index["<PAD>"], padding="post", maxlen=250)

# Write Models

Word embedding (?)

In [34]:
emb = keras.Sequential()
emb.add(keras.layers.Embedding(10000, 16))
emb.add(keras.layers.GlobalAveragePooling1D())
emb.add(keras.layers.Dense(16, activation="relu"))
emb.add(keras.layers.Dense(1, activation="sigmoid"))

emb.summary()
emb.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
emb.fit(x_train, y_train, epochs=40, batch_size=512, validation_split=0.3)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
35/35 [==============================] - 1s 21ms/step - loss: 0.6917 - accuracy: 0.6385 - val_loss: 0.6889 - val_accuracy: 0.7401
Epoch 2/40
35/35 [==============================] - 1s 18ms/step - loss: 0.6847 - accuracy: 0.7085 - val_

LSTM

In [35]:
lstm = keras.Sequential()
lstm.add(keras.layers.Embedding(num_words, 32))
lstm.add(keras.layers.LSTM(32))
lstm.add(keras.layers.Dense(1, activation='sigmoid'))
lstm.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)


Epoch 1/10
157/157 [==============================] - 26s 168ms/step - loss: 0.6850 - acc: 0.5339 - val_loss: 0.6600 - val_acc: 0.5620
Epoch 2/10
157/157 [==============================] - 26s 167ms/step - loss: 0.6365 - acc: 0.6549 - val_loss: 0.5300 - val_acc: 0.7712
Epoch 3/10
157/157 [==============================] - 26s 166ms/step - loss: 0.5356 - acc: 0.7538 - val_loss: 0.4885 - val_acc: 0.8144
Epoch 4/10
157/157 [==============================] - 26s 167ms/step - loss: 0.4963 - acc: 0.8015 - val_loss: 0.4486 - val_acc: 0.8250
Epoch 5/10
157/157 [==============================] - 26s 165ms/step - loss: 0.4198 - acc: 0.8427 - val_loss: 0.4462 - val_acc: 0.8346
Epoch 6/10
157/157 [==============================] - 28s 175ms/step - loss: 0.4090 - acc: 0.8497 - val_loss: 0.4165 - val_acc: 0.8414
Epoch 7/10
157/157 [==============================] - 29s 185ms/step - loss: 0.3738 - acc: 0.8680 - val_loss: 0.4165 - val_acc: 0.8400
Epoch 8/10
157/157 [==============================] - 2

GRU

In [36]:
gru = keras.Sequential()
gru.add(keras.layers.Embedding(num_words, 32))
gru.add(keras.layers.GRU(32))
gru.add(keras.layers.Dense(1, activation='sigmoid'))
gru.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 27s 171ms/step - loss: 0.3181 - acc: 0.8978 - val_loss: 0.3934 - val_acc: 0.8586
Epoch 2/10
157/157 [==============================] - 27s 172ms/step - loss: 0.3007 - acc: 0.9039 - val_loss: 0.3857 - val_acc: 0.8606
Epoch 3/10
157/157 [==============================] - 26s 166ms/step - loss: 0.2866 - acc: 0.9092 - val_loss: 0.6376 - val_acc: 0.7832
Epoch 4/10
157/157 [==============================] - 26s 166ms/step - loss: 0.2835 - acc: 0.9111 - val_loss: 0.4319 - val_acc: 0.8600
Epoch 5/10
157/157 [==============================] - 27s 170ms/step - loss: 0.3544 - acc: 0.8865 - val_loss: 0.3864 - val_acc: 0.8646
Epoch 6/10
157/157 [==============================] - 27s 170ms/step - loss: 0.2657 - acc: 0.9186 - val_loss: 0.4001 - val_acc: 0.8592
Epoch 7/10
157/157 [==============================] - 27s 169ms/step - loss: 0.2716 - acc: 0.9158 - val_loss: 0.4702 - val_acc: 0.8474
Epoch 8/10
157/157 [==============================] - 2

Simple RNN

In [45]:
srnn = keras.Sequential()
srnn.add(keras.layers.Embedding(num_words, 32))
srnn.add(keras.layers.SimpleRNN(32))
srnn.add(keras.layers.Dense(1, activation='sigmoid'))
srnn.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 12s 77ms/step - loss: 0.1856 - acc: 0.9255 - val_loss: 0.9398 - val_acc: 0.5994
Epoch 2/10
157/157 [==============================] - 12s 78ms/step - loss: 0.1768 - acc: 0.9299 - val_loss: 0.8908 - val_acc: 0.6284
Epoch 3/10
157/157 [==============================] - 12s 78ms/step - loss: 0.1891 - acc: 0.9232 - val_loss: 0.9724 - val_acc: 0.5958
Epoch 4/10
157/157 [==============================] - 12s 78ms/step - loss: 0.1612 - acc: 0.9356 - val_loss: 1.1960 - val_acc: 0.5382
Epoch 5/10
157/157 [==============================] - 12s 77ms/step - loss: 0.1611 - acc: 0.9351 - val_loss: 1.0697 - val_acc: 0.5840
Epoch 6/10
157/157 [==============================] - 12s 77ms/step - loss: 0.1261 - acc: 0.9539 - val_loss: 1.0480 - val_acc: 0.6134
Epoch 7/10
157/157 [==============================] - 12s 75ms/step - loss: 0.1562 - acc: 0.9381 - val_loss: 1.0684 - val_acc: 0.6064
Epoch 8/10
157/157 [==============================] - 12s 75ms

# Evaluate Models

Word embedding (?)

In [38]:
emb.evaluate(x_test, y_test)

782/782 [==============================] - 1s 1ms/step - loss: 0.3435 - accuracy: 0.8689


[0.34349995851516724, 0.8688799738883972]

LSTM

In [39]:
lstm.evaluate(x_test, y_test)

782/782 [==============================] - 17s 22ms/step - loss: 0.6932 - acc: 0.4966


[0.6932249069213867, 0.4965600073337555]

GRU

In [40]:
gru.evaluate(x_test, y_test)

782/782 [==============================] - 15s 19ms/step - loss: 0.6934 - acc: 0.4970


[0.6933724880218506, 0.4970400035381317]

Simple RNN

In [46]:
srnn.evaluate(x_test, y_test)

782/782 [==============================] - 10s 13ms/step - loss: 0.6977 - acc: 0.5006


[0.6976779103279114, 0.5005999803543091]